## **GENERADOR DE GRÁFICAS DE DSC**
___

### CARACTERÍSTICAS
y obtención de datos asociados a transformaciones y transiciones 
___

ESTRUCTURA DE ARCHIVO:

Tipo de datos aceptados: ASCII, codificación: ANSI
- Las columnas a leer son 5:
    1. Indice (ºC)
    2. Tiempo (s)
    3. Temperatura del horno (ºC)
    4. Temperatura de la muestra (ºC)
    5. Termogravimetría (mg)
    6. Flujo de calor (mW)

ESTRUCTURA DE NOMBRE:
* Nombre de archivo de curva a procesar: "nombre_fijo.txt"
* Nombre de archivo de curva instrumental: "Blanco_nombre_fijo.txt"

EJEMPLO:
* Curva a procesar: "Cp_Aluminio.txt"
* Curva instrumental: "Blanco_Cp_Aluminio.txt"

DETALLES:
1. Para los nombres utilizar como separador solo:       "_"
2. Usar mismo tiempo de adquisición en curva original e instrumental
3. Se debe cambiar las rutas de acceso donde el script leerá las curvas a procesar (experimental) y curvas blanco (instrumental)
4. Asegurarse de solo colocar los archivos a procesar en las carpetas correspondientes
5. Las carpetas deben existir

COMENTARIOS:
* El script ha sido probado solo para datos de DSC modelo Labsys evo


###  Importación de librerías necesarias
___

In [1]:
# Dependencies import
import os
import glob
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import pandas as pd
from pandas.errors import EmptyDataError


%matplotlib inline
# to make this notebook's output stable across runs
np.random.seed(42)

###  Lectura de las carpetas de trabajo y sus archivos correspondientes
___

In [2]:
# os.walk is a generator and calling next will get the first result in the form of a 3-tuple (dirpath, dirnames, filenames)
# Thus the [1] index returns only the dirnames from that tuple. The '.' is for root path of the project.
#next(os.walk('.'))[1]

dir_name = '.'
root, dirs, files = next(os.walk(dir_name))
if '.ipynb_checkpoints' in dirs:
    dirs.remove('.ipynb_checkpoints')

In [3]:
# get the index of 'experimental'
pos_experimental = dirs.index('experimental')
pos_blanco = dirs.index('blanco')

In [33]:
# especificamos un patrón de archivo y lo pasamos como parámetro en la función glob
files_exp = glob.glob(dirs[pos_experimental]+'\*.txt', recursive=False)

In [34]:
# especificamos un patrón de archivo y lo pasamos como parámetro en la función glob
files_bl = glob.glob(dirs[pos_blanco]+'\*.txt', recursive=False)

In [38]:
# guardamos la lista de archivos en una variable temporal, también de tipo lista
tmp = files_exp
tmp2 = []
only_name_exp = []

for fnam in tmp:
    tmp2.append(fnam.rsplit('.', 1)[0])

for fnam in tmp2:
    only_name_exp.append(fnam.rsplit('\\', 1)[1])

In [39]:
# mostramos la lista de nombres
for list_files in only_name_exp:
    print(list_files)

Ag_M1_Calentamiento_5x
Ag_M1_Enfriamiento_5x
Cd_M1_Calentamiento_10x
Cd_M1_Calentamiento_5x
Cd_M1_Enfriamiento_10x
Cd_M1_Enfriamiento_5x


In [ ]:
# guardamos la lista de archivos en una variable temporal, también de tipo lista
tmp = files_bl
tmp2 = []
only_name_bl = []

for fnam in tmp:
    tmp2.append(fnam.rsplit('.', 1)[0])

for fnam in tmp2:
    only_name_bl.append(fnam.rsplit('\\', 1)[1])

In [ ]:
# mostramos la lista de nombres
for list_files in only_name_bl:
    print(list_files)

###  De los archivos leídos, quitamos las columnas de datos que no utilizaremos
___

In [ ]:
# Create an empty list to store your data drames
#### result containers
good_files_exp = []
bad_files_exp = []

#### iterate files in directory
for one_file in files_exp:
    print(f"Parsing: {one_file}")
    try:
        #### read csv file to pandas data frame
        df = pd.read_table(one_file, encoding='UTF-16LE', skiprows=14, delim_whitespace=True, header=None)
        #### drop the data we'll not use
        head = ["idx", "t", "Th", "Tm", "TG", "f"]
        df.columns = head
        df.drop('TG', inplace=True, axis=1)
        df.drop('idx', inplace=True, axis=1)
        df.drop('Th', inplace=True, axis=1)
        #### append the file name to a list of good files
        good_files_exp.append(df)
        # ...
    except EmptyDataError:
        print(f"No columns to parse from file {one_file}")
        bad_files_exp.append(one_file)
print('Done parsing.')

In [42]:
# mostramos los primeros 5 elementos de una tabla como ejemplo
print(good_files_exp[0].head(5))

     t         Tm          f
0  0.5  89.123482 -17.413842
1  1.0  89.145005 -17.405782
2  1.5  89.169185 -17.392898
3  2.0  89.203026 -17.377975
4  2.5  89.237086 -17.365451


In [37]:
# Create an empty list to store your data drames
#### result containers
good_files_bl = []
bad_files_bl = []

#### iterate files in directory
for one_file in files_exp:
    print(f"Parsing: {one_file}")
    try:
        #### read csv file to pandas data frame
        df = pd.read_table(one_file, encoding='UTF-16LE', skiprows=14, delim_whitespace=True, header=None)
        #### drop the data we'll not use
        head = ["idx", "t", "Th", "Tm", "TG", "f"]
        df.columns = head
        df.drop('TG', inplace=True, axis=1)
        df.drop('idx', inplace=True, axis=1)
        df.drop('Th', inplace=True, axis=1)
        #### append the file name to a list of good files
        good_files_bl.append(df)
        # ...
    except EmptyDataError:
        print(f"No columns to parse from file {one_file}")
        bad_files_bl.append(one_file)
print('Done parsing.')

Parsing: experimental\Ag_M1_Calentamiento_5x.txt
Parsing: experimental\Ag_M1_Enfriamiento_5x.txt
Parsing: experimental\Cd_M1_Calentamiento_10x.txt
Parsing: experimental\Cd_M1_Calentamiento_5x.txt
Parsing: experimental\Cd_M1_Enfriamiento_10x.txt
Parsing: experimental\Cd_M1_Enfriamiento_5x.txt
Done parsing.


In [ ]:
# mostramos los primeros 5 elementos de una tabla como ejemplo
print(good_files_bl[0].head(5))

In [43]:
# creamos variable para iterar
i = 0
for file_to_graph in good_files_exp:
    print(f"Graphing ΦQ (mW) vs t (s): {only_name_exp[i]}_vs_t")
    # Gráficos de flujo de calor vs temperatura de muestra
    fig1 = plt.figure(figsize=(12, 7.68))
    fig1.patch.set_facecolor("#6D9EC1")
    fig1.patch.set_alpha(0.15)

    ax = fig1.add_subplot()
    ax.patch.set_facecolor("#6D9EC1")

    ax.plot(file_to_graph.t, file_to_graph.f, linestyle='-', linewidth=0.5, color = "white", label="dT")
    ax.set_ylabel('ΦQ (mW)')
    ax.set_xlabel('t (s)')
    ax.legend();
    
    fig1.savefig(f'graficos\{only_name_exp[i]}_vs_t.png')
    plt.close()
    i+=1
print('Done graphing.')

Graphing ΦQ (mW) vs t (s): Ag_M1_Calentamiento_5x_vs_t
Graphing ΦQ (mW) vs t (s): Ag_M1_Enfriamiento_5x_vs_t
Graphing ΦQ (mW) vs t (s): Cd_M1_Calentamiento_10x_vs_t
Graphing ΦQ (mW) vs t (s): Cd_M1_Calentamiento_5x_vs_t
Graphing ΦQ (mW) vs t (s): Cd_M1_Enfriamiento_10x_vs_t
Graphing ΦQ (mW) vs t (s): Cd_M1_Enfriamiento_5x_vs_t
Done graphing.


In [44]:
# creamos variable para iterar
i = 0
for file_to_graph in good_files:
    print(f"Graphing ΦQ (mW) vs T (°C): {only_name_bl[i]}_vs_Tm")
    # Gráficos de flujo de calor vs temperatura de muestra
    fig2 = plt.figure(figsize=(12, 7.68))
    fig2.patch.set_facecolor("#6D9EC1")
    fig2.patch.set_alpha(0.15)

    ax = fig2.add_subplot()
    ax.patch.set_facecolor("#6D9EC1")

    ax.plot(file_to_graph.Tm, file_to_graph.f, linestyle='-', linewidth=0.5, color = "white", label="dT")
    ax.set_ylabel('ΦQ (mW)')
    ax.set_xlabel('T (°C)')
    ax.legend();
    
    fig2.savefig(f'graficos\{only_name_bl[i]}_vs_Tm.png')
    plt.close()
    i+=1
print('Done graphing.')

Graphing ΦQ (mW) vs T (°C): Ag_M1_Calentamiento_5x_vs_Tm
Graphing ΦQ (mW) vs T (°C): Ag_M1_Enfriamiento_5x_vs_Tm
Graphing ΦQ (mW) vs T (°C): Cd_M1_Calentamiento_10x_vs_Tm
Graphing ΦQ (mW) vs T (°C): Cd_M1_Calentamiento_5x_vs_Tm
Graphing ΦQ (mW) vs T (°C): Cd_M1_Enfriamiento_10x_vs_Tm
Graphing ΦQ (mW) vs T (°C): Cd_M1_Enfriamiento_5x_vs_Tm
Done graphing.


In [ ]:
------------------------------------------------------------------
#  Creación de celdas para almacenar las diferentes curvas generadas
------------------------------------------------------------------

#   Para almacenar la sustracción de la curva instrumental de la curva
#   experimental
Substracted_EXP = cell(1, co);
#   Para almacenar la sustracción de la curva instrumental de la curva
#   de material de referencia
Substracted_REF = cell(1, co);
#   Para almacenar la curva experimental suavizada y sustraída por la
#   curva instrumental
Smooth_Flux_Curves = cell(1, co);
#   Para almacenar el tiempo de adquisición
TimeStep = cell(1, co);
#   Para almacenar el índice de los datos del punto de límite inferior
Idx_Amin = cell(1, co);
#   Para almacenar el índice de los datos del punto de límite superior
Idx_Amax = cell(1, co);
#   Para almacenar el índice de los datos del punto de límite inferior
Idx_Bmin = cell(1, co);
#   Para almacenar el índice de los datos del punto de límite superior
Idx_Bmax = cell(1, co);
#   Para almacenar el índice de los datos del punto de límite inferior
Idx_Cmin = cell(1, co);
#   Para almacenar el índice de los datos del punto de límite superior
Idx_Cmax = cell(1, co);
#   Masas como constantes, cambiar según corresponda
mref = 37.52;
mexp = 25.59;
#   Para almacenar el resultado de Cp del material a medir
Cexp = cell(1, co);
#   Para almacenar el resultado de Cp cero corregido del material a medir
Cexp0 = cell(1, co);
#   Para almacenar los coeficientes de regresión de la curva de Cp
coef = cell(1, co);

#   Se obtiene tamaño de la pantalla, scrsz = [left, bottom, width, height]
scrsz = get(0,'ScreenSize');

------------------------------------------------------------------
#  Bucle de procesamiento de archivos, generación de gráficas y datos
------------------------------------------------------------------

for i = 1:co
    
    Q = 1;
    while Q == 1
        
        ---------------------------------------------------------------
        #  Grafica provisoria para visualizar la curva completa del
        #  experimento, Q (flujo de calor) vs t (tiempo)
        ---------------------------------------------------------------
        
        #   Se genera el título para el gráfico
        GraphTitle = [EXP_GraphNames{i} ' - Q vs t'];
        #   Se genera una figura nueva para trabajar con las gráficas
        figure('Name',GraphTitle,'OuterPosition',scrsz);
        #   Se crea una grafica sobre la figura actual
        plot(EXP_FILES_cell{i}(:, 2), EXP_FILES_cell{i}(:, 6),...
            'LineWidth',2);
        #   Cambio de formato del texto del título
        title(GraphTitle,'FontSize',24,'FontWeight','bold','Color','black');
        #   Cambio de formato del texto de la ordenada
        xlabel('Tiempo (s)','FontSize',18,'FontWeight','bold','Color','black');
        #   Cambio de formato del texto de la absisa
        ylabel('Flujo de Calor (mW)','FontSize',18,'FontWeight','bold',...
            'Color','blue');
        set(gca,'FontSize',16,'XColor','black','YColor','blue');
        legend(EXP_GraphNames{i}, 'Location','Best');
            #'TickDir','out','TickLength',[.2,.2],'YLim',[-2,2]);
        
        #   Se selecciona el directorio para almacenar las gráficas
        cd(DIR_graficas);
        #   Se almacena la gráfica
        saveas(1, GraphTitle, 'fig');
        saveas(1, GraphTitle, 'bmp');
        
        #   Se selecciona el directorio para almacenar los datos
        cd(DIR_datos);
        writetable(table(EXP_FILES_cell{i}(:, 2),...
            EXP_FILES_cell{i}(:, 6)),...
            [GraphTitle '.dat'],'Delimiter','\t','WriteRowNames',true);
        #   Se congela la figura para seleccionar los puntos del gráfico
        pause;
        
        -----------------------------------------------------------
        #  Selección manual de la sección del gráfico dónde ocurre
        #  la transformación que se pretende procesar
        -----------------------------------------------------------
        
        uiwait(msgbox('Selección de la seccion donde se desea trabajar.',...
            'Success','modal'));
        #   Obtención de los puntos seleccionados en el gráfico
        A = ginput(2);
        
        uiwait(msgbox('Selección del punto de fin de la isoterma previa y de inicio de la sisoterma posterior a la zona de trabajo.' ...
            , 'Success','modal'));
        #   Obtención de los puntos seleccionados en el gráfico
        B = ginput(2);
        
        #   Cierre de la ventana de la figura
        close;
        
        -----------------------------------------------------------
        #  Obtención del tiempo de adquisición, datos por segundo
        -----------------------------------------------------------
        
        #   Se obtienen dimensiones de la celda donde se encuentran los
        #   archivos, siendo Fil: fila y Col: columna
        [Fil, Col] = size(EXP_FILES_cell{i});
        #   Se suman todos los datos
        for  k = 1:Fil
            TimeStep{i} =+ EXP_FILES_cell{i}(k, 2);
        end
        #   Se divide por la cantidad de datos
        TimeStep{i} = TimeStep{i} / Fil;
        #   Se obtienen los datos por segundo
        TimeStep{i} = 1 / TimeStep{i};
        
        -----------------------------------------------------------
        #  Búsqueda de la posición de los puntos seleccionados 
        #  manualmente mediante el índice de datos del archivo
        -----------------------------------------------------------
        
        Idx_Amin{i} = A(1,1) * TimeStep{i};
        Idx_Amin{i} = round(Idx_Amin{i});
        Idx_Amax{i} = A(2,1) * TimeStep{i};
        Idx_Amax{i} = round(Idx_Amax{i});
        
        Idx_Bmin{i} = B(1,1) * TimeStep{i};
        Idx_Bmin{i} = round(Idx_Bmin{i});
        Idx_Bmax{i} = B(2,1) * TimeStep{i};
        Idx_Bmax{i} = round(Idx_Bmax{i});
        
        -----------------------------------------------------------
        #  Gráfica de Q vs T de la zona seleccionada
        -----------------------------------------------------------
        
        #   Se genera el título para el gráfico
        GraphTitle = [EXP_GraphNames{i} ' - Q vs T - work zone'];
        #   Se genera una figura nueva para trabajar con las gráficas
        figure('Name',GraphTitle,'OuterPosition',scrsz);
        #   Se crea una grafica sobre la figura actual
        plot(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4),...
            EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 6),'LineWidth',2);
        #   Cambio de formato del texto del título
        title(GraphTitle,'FontSize',24,'FontWeight','bold','Color','black');
        #   Cambio de formato del texto de la ordenada
        xlabel('T (°C)','FontSize',18,'FontWeight','bold','Color','black');
        #   Cambio de formato del texto de la absisa
        ylabel('Flujo de Calor (mW)','FontSize',18,'FontWeight','bold',...
            'Color','blue');
        set(gca,'FontSize',16,'XColor','black','YColor','blue');
        legend(EXP_GraphNames{i}, 'Location','Best');
        #   Se congela la figura para poder observar
        pause;
        
        -----------------------------------------------------------
        #  Diálogo de consulta
        -----------------------------------------------------------

        #   Apertura de cuadro de diálogo,
        Qst = questdlg('¿Es la zona del gráfico donde desea trabajar?',...
            'Confirme para continuar','Si, continuar',...
            'No, volver a seleccionar','Salir','Salir');
        switch Qst
            case 'Si, continuar'
                Q = 0;
                
                #   Se selecciona el directorio para almacenar las gráficas
                cd(DIR_graficas);
                #   Se almacena la gráfica
                saveas(1, GraphTitle, 'fig');
                saveas(1, GraphTitle, 'bmp');
                
                #   Se selecciona el directorio para almacenar los datos
                cd(DIR_datos);
                writetable(table(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 2),...
                    EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 6)),...
                    [GraphTitle '.dat'],'Delimiter','\t','WriteRowNames',true);
                
                #   Cierre de la ventana de la figura
                close;
            case 'No, volver a seleccionar'
                Q = 1;
                #   Cierre de la ventana de la figura
                close;
            case 'Salir'
                quit;
        end
        
        ---------------------------------------------------------------
        #  Fit de los flujos de calor para la ecuacion de Cp
        ---------------------------------------------------------------
        
        p1 = polyfit(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4), ...
            EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 6), 7);
        polyEXP = polyval(p1, EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4));
        
        p2 = polyfit(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4), ...
            INST_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 6), 7);
        polyINST = polyval(p2, EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4));
        
        p3 = polyfit(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4), ...
            REF_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 6), 7);
        polyREF = polyval(p3, EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4));
        
        p4 = polyfit(CpREF_cell{1}(:, 1), CpREF_cell{1}(:, 2), 7);
        polyCpREF = polyval(p4, EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4));
        
        ---------------------------------------------------------------
        #  Sustracción de la curva instrumental de la curva a experimental
        ---------------------------------------------------------------
        
        Substracted_EXP = polyEXP - polyINST;
        Substracted_REF = polyREF - polyINST;
        
        ---------------------------------------------------------------
        #  Construcción de la curva de Cp del material utilizado
        ---------------------------------------------------------------
        
        q1 = Substracted_EXP - Substracted_REF;
        Cexp{i} = q1 * ( mref / mexp ) .* polyCpREF;
        
        Cexp{i} = smooth( Cexp{i}, 75);
        
        T = (EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4) + 273.15) / 1000;
        Ccorr = Cexp{i} / (mexp / 26.9815386);
        p5 = polyfit (T, Ccorr, 3);
        coef{i} = [num2str(p5(1)) 'x^3 +' num2str(p5(2)) 'x^2 +' num2str(p5(3)) 'x +' num2str(p5(4))];
        
        -----------------------------------------------------------
        #  Gráfica de Cp cero corregido vs T de la zona seleccionada
        -----------------------------------------------------------
        
        #   Se genera el título para el gráfico
        GraphTitle = [EXP_GraphNames{i} ' - Cp vs T'];
        #   Se genera una figura nueva para trabajar con las gráficas
        figure('Name',GraphTitle,'OuterPosition',scrsz);
        #   Se crea una grafica sobre la figura actual
        plot(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4), Cexp{i},'LineWidth',2);
        #   Cambio de formato del texto del título
        title(GraphTitle,'FontSize',24,'FontWeight','bold','Color','black');
        #   Cambio de formato del texto de la ordenada
        xlabel('T (°C)','FontSize',18,'FontWeight','bold','Color','black');
        #   Cambio de formato del texto de la absisa
        ylabel('Cp (J/g°C)','FontSize',18,'FontWeight','bold','Color','blue');
        set(gca,'FontSize',16,'XColor','black','YColor','blue');
        legend(EXP_GraphNames{i}, 'Location','Best');
        #   Se selecciona el directorio para almacenar las gráficas
        cd(DIR_graficas);
        #   Se almacena la gráfica
        saveas(1, GraphTitle, 'fig');
        saveas(1, GraphTitle, 'bmp');
        
        #   Se selecciona el directorio para almacenar los datos
        cd(DIR_datos);
        writetable(table(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4), Cexp{i}),...
            [GraphTitle '.dat'],'Delimiter','\t','WriteRowNames',true);
        #   Cierre de la ventana de la figura
        close;
        
        ---------------------------------------------------------------
        #  Correción cero teórico de las curvas experimentales
        ---------------------------------------------------------------
        
        Correction_EXP = EXP_FILES_cell{i}(Idx_Bmin{i}, 6) + ((EXP_FILES_cell{i}(Idx_Bmax{i}, 6) - EXP_FILES_cell{i}(Idx_Bmin{i}, 6)) / (EXP_FILES_cell{i}(Idx_Bmax{i}, 2) - EXP_FILES_cell{i}(Idx_Bmin{i}, 2))) * (EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 2)- Idx_Bmin{i}(1, 1));
        Correction_REF = REF_FILES_cell{i}(Idx_Bmin{i}, 6) + ((REF_FILES_cell{i}(Idx_Bmax{i}, 6) - REF_FILES_cell{i}(Idx_Bmin{i}, 6)) / (REF_FILES_cell{i}(Idx_Bmax{i}, 2) - REF_FILES_cell{i}(Idx_Bmin{i}, 2))) * (REF_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 2)- Idx_Bmin{i}(1, 1));
        Correction_INST = INST_FILES_cell{i}(Idx_Bmin{i}, 6) + ((INST_FILES_cell{i}(Idx_Bmax{i}, 6) - INST_FILES_cell{i}(Idx_Bmin{i}, 6)) / (INST_FILES_cell{i}(Idx_Bmax{i}, 2) - INST_FILES_cell{i}(Idx_Bmin{i}, 2))) * (INST_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 2)- Idx_Bmin{i}(1, 1));
        
        Corrected_EXP = polyEXP - Correction_EXP;
        Corrected_REF = polyREF - Correction_REF;
        Corrected_INST = polyINST - Correction_INST;
        
        ---------------------------------------------------------------
        #  Sustracción de la curva instrumental de la curva a experimental
        ---------------------------------------------------------------
        
        Substracted_EXP = Corrected_EXP - Corrected_INST;
        Substracted_REF = Corrected_REF - Corrected_INST;
        
        ---------------------------------------------------------------
        #  Construcción de la curva de Cp del material utilizado
        ---------------------------------------------------------------
        
        q2 = Substracted_EXP ./ Substracted_REF;
        Cexp0{i} = q2 * ( mref / mexp ) .* polyCpREF;
        
        Cexp0{i} = smooth( Cexp0{i}, 75);
        
        -----------------------------------------------------------
        #  Gráfica de Cp cero corregido vs T de la zona seleccionada
        -----------------------------------------------------------
        
        #   Se genera el título para el gráfico
        GraphTitle = [EXP_GraphNames{i} ' - Cp0 vs T'];
        #   Se genera una figura nueva para trabajar con las gráficas
        figure('Name',GraphTitle,'OuterPosition',scrsz);
        #   Se crea una grafica sobre la figura actual
        plot(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4), Cexp0{i},'LineWidth',2);
        #   Cambio de formato del texto del título
        title(GraphTitle,'FontSize',24,'FontWeight','bold','Color','black');
        #   Cambio de formato del texto de la ordenada
        xlabel('T (°C)','FontSize',18,'FontWeight','bold','Color','black');
        #   Cambio de formato del texto de la absisa
        ylabel('Cp (J/g°C)','FontSize',18,'FontWeight','bold','Color','blue');
        set(gca,'FontSize',16,'XColor','black','YColor','blue');
        legend(EXP_GraphNames{i}, 'Location','Best');
        #   Se selecciona el directorio para almacenar las gráficas
        cd(DIR_graficas);
        #   Se almacena la gráfica
        saveas(1, GraphTitle, 'fig');
        saveas(1, GraphTitle, 'bmp');
        
        #   Se selecciona el directorio para almacenar los datos
        cd(DIR_datos);
        writetable(table(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4), Cexp0{i}),...
            [GraphTitle '.dat'],'Delimiter','\t','WriteRowNames',true);
        #   Cierre de la ventana de la figura
        close;
    end
end

        -----------------------------------------------------------
        #  Grafica provisoria para visualizar la curva completa del
        #  experimento, Q (flujo de calor) vs t (tiempo)
        ---------------------------------------------------------------
        
        #   Se genera el título para el gráfico
        GraphTitle = [EXP_GraphNames{i} ' - Q vs t'];
        #   Se genera una figura nueva para trabajar con las gráficas
        figure('Name',GraphTitle,'OuterPosition',scrsz);
        #   Se crea una grafica sobre la figura actual
        plot(EXP_FILES_cell{i}(:, 2), EXP_FILES_cell{i}(:, 6),...
            'LineWidth',2);
        #   Cambio de formato del texto del título
        title(GraphTitle,'FontSize',24,'FontWeight','bold','Color','black');
        #   Cambio de formato del texto de la ordenada
        xlabel('Tiempo (s)','FontSize',18,'FontWeight','bold','Color','black');
        #   Cambio de formato del texto de la absisa
        ylabel('Flujo de Calor (mW)','FontSize',18,'FontWeight','bold',...
            'Color','blue');
        set(gca,'FontSize',16,'XColor','black','YColor','blue');
        legend(EXP_GraphNames{i}, 'Location','Best');
            #'TickDir','out','TickLength',[.2,.2],'YLim',[-2,2]);
        
        #   Se selecciona el directorio para almacenar las gráficas
        cd(DIR_graficas);
        #   Se almacena la gráfica
        saveas(1, GraphTitle, 'fig');
        saveas(1, GraphTitle, 'bmp');
        
        #   Se selecciona el directorio para almacenar los datos
        cd(DIR_datos);
        writetable(table(EXP_FILES_cell{i}(:, 2),...
            EXP_FILES_cell{i}(:, 6)),...
            [GraphTitle '.dat'],'Delimiter','\t','WriteRowNames',true);
        #   Se congela la figura para seleccionar los puntos del gráfico
        pause;
        
        -----------------------------------------------------------
        Selección manual de la sección del gráfico dónde ocurre
        la transformación que se pretende procesar
        -----------------------------------------------------------
        
        uiwait(msgbox('Selección de la seccion donde se desea trabajar.',...
            'Success','modal'));
        #   Obtención de los puntos seleccionados en el gráfico
        A = ginput(2);
        
        uiwait(msgbox('Selección del punto de fin de la isoterma previa y de inicio de la sisoterma posterior a la zona de trabajo.' ...
            , 'Success','modal'));
        #   Obtención de los puntos seleccionados en el gráfico
        B = ginput(2);
        
        #   Cierre de la ventana de la figura
        close;
        
        -----------------------------------------------------------
        Obtención del tiempo de adquisición, datos por segundo
        -----------------------------------------------------------
        
        #   Se obtienen dimensiones de la celda donde se encuentran los
        #   archivos, siendo Fil: fila y Col: columna
        [Fil, Col] = size(EXP_FILES_cell{i});
        #   Se suman todos los datos
        for  k = 1:Fil
            TimeStep{i} =+ EXP_FILES_cell{i}(k, 2);
        end
        #   Se divide por la cantidad de datos
        TimeStep{i} = TimeStep{i} / Fil;
        #   Se obtienen los datos por segundo
        TimeStep{i} = 1 / TimeStep{i};
        
        -----------------------------------------------------------
        Búsqueda de la posición de los puntos seleccionados 
        manualmente mediante el índice de datos del archivo
        -----------------------------------------------------------
        
        Idx_Amin{i} = A(1,1) * TimeStep{i};
        Idx_Amin{i} = round(Idx_Amin{i});
        Idx_Amax{i} = A(2,1) * TimeStep{i};
        Idx_Amax{i} = round(Idx_Amax{i});
        
        Idx_Bmin{i} = B(1,1) * TimeStep{i};
        Idx_Bmin{i} = round(Idx_Bmin{i});
        Idx_Bmax{i} = B(2,1) * TimeStep{i};
        Idx_Bmax{i} = round(Idx_Bmax{i});
        
        -----------------------------------------------------------
        Gráfica de Q vs T de la zona seleccionada
        -----------------------------------------------------------
        
        #   Se genera el título para el gráfico
        GraphTitle = [EXP_GraphNames{i} ' - Q vs T - work zone'];
        #   Se genera una figura nueva para trabajar con las gráficas
        figure('Name',GraphTitle,'OuterPosition',scrsz);
        #   Se crea una grafica sobre la figura actual
        plot(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4),...
            EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 6),'LineWidth',2);
        #   Cambio de formato del texto del título
        title(GraphTitle,'FontSize',24,'FontWeight','bold','Color','black');
        #   Cambio de formato del texto de la ordenada
        xlabel('T (°C)','FontSize',18,'FontWeight','bold','Color','black');
        #   Cambio de formato del texto de la absisa
        ylabel('Flujo de Calor (mW)','FontSize',18,'FontWeight','bold',...
            'Color','blue');
        set(gca,'FontSize',16,'XColor','black','YColor','blue');
        legend(EXP_GraphNames{i}, 'Location','Best');
        #   Se congela la figura para poder observar
        pause;
        
        -----------------------------------------------------------
        Diálogo de consulta
        -----------------------------------------------------------

        #   Apertura de cuadro de diálogo,
        Qst = questdlg('¿Es la zona del gráfico donde desea trabajar?',...
            'Confirme para continuar','Si, continuar',...
            'No, volver a seleccionar','Salir','Salir');
        switch Qst
            case 'Si, continuar'
                Q = 0;
                
                #   Se selecciona el directorio para almacenar las gráficas
                cd(DIR_graficas);
                #   Se almacena la gráfica
                saveas(1, GraphTitle, 'fig');
                saveas(1, GraphTitle, 'bmp');
                
                #   Se selecciona el directorio para almacenar los datos
                cd(DIR_datos);
                writetable(table(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 2),...
                    EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 6)),...
                    [GraphTitle '.dat'],'Delimiter','\t','WriteRowNames',true);
                
                #   Cierre de la ventana de la figura
                close;
            case 'No, volver a seleccionar'
                Q = 1;
                #   Cierre de la ventana de la figura
                close;
            case 'Salir'
                quit;
        end
        
        ---------------------------------------------------------------
        Fit de los flujos de calor para la ecuacion de Cp
        ---------------------------------------------------------------
        
        p1 = polyfit(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4), ...
            EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 6), 7);
        polyEXP = polyval(p1, EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4));
        
        p2 = polyfit(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4), ...
            INST_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 6), 7);
        polyINST = polyval(p2, EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4));
        
        p3 = polyfit(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4), ...
            REF_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 6), 7);
        polyREF = polyval(p3, EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4));
        
        p4 = polyfit(CpREF_cell{1}(:, 1), CpREF_cell{1}(:, 2), 7);
        polyCpREF = polyval(p4, EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4));
        
        ---------------------------------------------------------------
        Sustracción de la curva instrumental de la curva a experimental
        ---------------------------------------------------------------
        
        Substracted_EXP = polyEXP - polyINST;
        Substracted_REF = polyREF - polyINST;
        
        ---------------------------------------------------------------
        Construcción de la curva de Cp del material utilizado
        ---------------------------------------------------------------
        
        q1 = Substracted_EXP - Substracted_REF;
        Cexp{i} = q1 * ( mref / mexp ) .* polyCpREF;
        
        Cexp{i} = smooth( Cexp{i}, 75);
        
        T = (EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4) + 273.15) / 1000;
        Ccorr = Cexp{i} / (mexp / 26.9815386);
        p5 = polyfit (T, Ccorr, 3);
        coef{i} = [num2str(p5(1)) 'x^3 +' num2str(p5(2)) 'x^2 +' num2str(p5(3)) 'x +' num2str(p5(4))];
        
        -----------------------------------------------------------
        Gráfica de Cp cero corregido vs T de la zona seleccionada
        -----------------------------------------------------------
        
        #   Se genera el título para el gráfico
        GraphTitle = [EXP_GraphNames{i} ' - Cp vs T'];
        #   Se genera una figura nueva para trabajar con las gráficas
        figure('Name',GraphTitle,'OuterPosition',scrsz);
        #   Se crea una grafica sobre la figura actual
        plot(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4), Cexp{i},'LineWidth',2);
        #   Cambio de formato del texto del título
        title(GraphTitle,'FontSize',24,'FontWeight','bold','Color','black');
        #   Cambio de formato del texto de la ordenada
        xlabel('T (°C)','FontSize',18,'FontWeight','bold','Color','black');
        #   Cambio de formato del texto de la absisa
        ylabel('Cp (J/g°C)','FontSize',18,'FontWeight','bold','Color','blue');
        set(gca,'FontSize',16,'XColor','black','YColor','blue');
        legend(EXP_GraphNames{i}, 'Location','Best');
        #   Se selecciona el directorio para almacenar las gráficas
        cd(DIR_graficas);
        #   Se almacena la gráfica
        saveas(1, GraphTitle, 'fig');
        saveas(1, GraphTitle, 'bmp');
        
        #   Se selecciona el directorio para almacenar los datos
        cd(DIR_datos);
        writetable(table(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4), Cexp{i}),...
            [GraphTitle '.dat'],'Delimiter','\t','WriteRowNames',true);
        #   Cierre de la ventana de la figura
        close;
        
        ---------------------------------------------------------------
        Correción cero teórico de las curvas experimentales
        ---------------------------------------------------------------
        
        Correction_EXP = EXP_FILES_cell{i}(Idx_Bmin{i}, 6) + ((EXP_FILES_cell{i}(Idx_Bmax{i}, 6) - EXP_FILES_cell{i}(Idx_Bmin{i}, 6)) / (EXP_FILES_cell{i}(Idx_Bmax{i}, 2) - EXP_FILES_cell{i}(Idx_Bmin{i}, 2))) * (EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 2)- Idx_Bmin{i}(1, 1));
        Correction_REF = REF_FILES_cell{i}(Idx_Bmin{i}, 6) + ((REF_FILES_cell{i}(Idx_Bmax{i}, 6) - REF_FILES_cell{i}(Idx_Bmin{i}, 6)) / (REF_FILES_cell{i}(Idx_Bmax{i}, 2) - REF_FILES_cell{i}(Idx_Bmin{i}, 2))) * (REF_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 2)- Idx_Bmin{i}(1, 1));
        Correction_INST = INST_FILES_cell{i}(Idx_Bmin{i}, 6) + ((INST_FILES_cell{i}(Idx_Bmax{i}, 6) - INST_FILES_cell{i}(Idx_Bmin{i}, 6)) / (INST_FILES_cell{i}(Idx_Bmax{i}, 2) - INST_FILES_cell{i}(Idx_Bmin{i}, 2))) * (INST_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 2)- Idx_Bmin{i}(1, 1));
        
        Corrected_EXP = polyEXP - Correction_EXP;
        Corrected_REF = polyREF - Correction_REF;
        Corrected_INST = polyINST - Correction_INST;
        
        ---------------------------------------------------------------
        Sustracción de la curva instrumental de la curva a experimental
        ---------------------------------------------------------------
        
        Substracted_EXP = Corrected_EXP - Corrected_INST;
        Substracted_REF = Corrected_REF - Corrected_INST;
        
        ---------------------------------------------------------------
        Construcción de la curva de Cp del material utilizado
        ---------------------------------------------------------------
        
        q2 = Substracted_EXP ./ Substracted_REF;
        Cexp0{i} = q2 * ( mref / mexp ) .* polyCpREF;
        
        Cexp0{i} = smooth( Cexp0{i}, 75);
        
        -----------------------------------------------------------
        Gráfica de Cp cero corregido vs T de la zona seleccionada
        -----------------------------------------------------------
        
        #   Se genera el título para el gráfico
        GraphTitle = [EXP_GraphNames{i} ' - Cp0 vs T'];
        #   Se genera una figura nueva para trabajar con las gráficas
        figure('Name',GraphTitle,'OuterPosition',scrsz);
        #   Se crea una grafica sobre la figura actual
        plot(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4), Cexp0{i},'LineWidth',2);
        #   Cambio de formato del texto del título
        title(GraphTitle,'FontSize',24,'FontWeight','bold','Color','black');
        #   Cambio de formato del texto de la ordenada
        xlabel('T (°C)','FontSize',18,'FontWeight','bold','Color','black');
        #   Cambio de formato del texto de la absisa
        ylabel('Cp (J/g°C)','FontSize',18,'FontWeight','bold','Color','blue');
        set(gca,'FontSize',16,'XColor','black','YColor','blue');
        legend(EXP_GraphNames{i}, 'Location','Best');
        #   Se selecciona el directorio para almacenar las gráficas
        cd(DIR_graficas);
        #   Se almacena la gráfica
        saveas(1, GraphTitle, 'fig');
        saveas(1, GraphTitle, 'bmp');
        
        #   Se selecciona el directorio para almacenar los datos
        cd(DIR_datos);
        writetable(table(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4), Cexp0{i}),...
            [GraphTitle '.dat'],'Delimiter','\t','WriteRowNames',true);
        #   Cierre de la ventana de la figura
        close;
    end
end

-----------------------------------------------------------
Gráfica de Cp vs T de la zona seleccionada
-----------------------------------------------------------

#   Se genera el título para el gráfico
GraphTitle = [EXP_GraphNames{i} ' - Cp vs T - all'];
#   Se genera una figura nueva para trabajar con las gráficas
figure('Name',GraphTitle,'OuterPosition',scrsz);

for i = 1:co
    
    #   Se crea una grafica sobre la figura actual
    hold all;
    plot(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4), Cexp{i}, 'LineWidth',2);
    
end

for i = 1:co
    
    #   Se crea una grafica sobre la figura actual
    hold all;
    plot(EXP_FILES_cell{i}(Idx_Amin{i}:Idx_Amax{i}, 4), Cexp0{i}, 'LineWidth',2);
    
end

#   Cambio de formato del texto del título
title(GraphTitle,'FontSize',24,'FontWeight','bold','Color','black');
#   Cambio de formato del texto de la ordenada
xlabel('T (°C)','FontSize',18,'FontWeight','bold','Color','black');
#   Cambio de formato del texto de la absisa
ylabel('Cp (J/g°C)','FontSize',18,'FontWeight','bold','Color','blue');
set(gca,'FontSize',16,'XColor','black');
legend([EXP_GraphNames, EXP_GraphNames], 'Location','Best');
#   Se congela la figura para poder observar
pause;
#   Se selecciona el directorio para almacenar las gráficas
cd(DIR_graficas);
#   Se almacena la gráfica
saveas(1, GraphTitle, 'fig');
saveas(1, GraphTitle, 'bmp');

#   Cierre de la ventana de la figura
close;